In [1]:
import sys
import os
from datetime import datetime 

import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction import DictVectorizer 
import xgboost as xgb

import mlflow

from misc import init

In [2]:
# initialize configuration
CONFIG = init.init_config()

Missing the following settings: TARGET, MLFLOW_DB_PATH, MLFLOW_EXPERIMENT_NAME, PARAMS


In [3]:
# read data
data = pd.read_parquet(CONFIG["DATA_PATH"])

In [14]:
# setup training context
# get datasets
xtrain, xtest = train_test_split(data, test_size=0.2, random_state=99)

# get targets
ytrain = xtrain[CONFIG["TARGET"]].values
ytest = xtest[CONFIG["TARGET"]].values
xtrain.drop(columns=[CONFIG["TARGET"]], inplace=True)
xtest.drop(columns=[CONFIG["TARGET"]], inplace=True)

# vectorize
dv = DictVectorizer(sparse=False)
train_dict = xtrain.to_dict(orient="records")
test_dict = xtest.to_dict(orient="records")
xtrain = dv.fit_transform(train_dict)
xtest = dv.transform(test_dict)
feature_names = dv.get_feature_names_out().tolist()

# get dmatrix
xtrain = xgb.DMatrix(xtrain, ytrain, feature_names=feature_names)
xtest = xgb.DMatrix(xtest, ytest, feature_names=feature_names)

In [ ]:
# # launch mlflow
# mlflow.set_tracking_uri(f"sqlite:///{CONFIG["MLFLOW_DB_PATH"]}")
# mlflow.set_experiment()

In [5]:
l = ["a", "b", "c"]
", ".join(l)

'a, b, c'